In [1]:
%matplotlib widget
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt
import matplotlib.dates as mdates

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float


In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
inspector = inspect(engine)
inspector.get_table_names()




# reflect the tables

['measurement', 'station']

In [55]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])


columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [9]:
# We can view all of the classes that automap found
Base = automap_base()
Base.prepare(engine, reflect=True)



In [10]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [11]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
last_date = session.query(func.max(Measurement.date)).all()





# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [44]:
# Calculate the date 1 year ago from the last data point in the database
# Perform a query to retrieve the data and precipitation scores
last12 = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date >= '2016-08-24').group_by(Measurement.date).all()
# Save the query results as a Pandas DataFrame and set the index to the date column
df= pd.DataFrame(last12,columns=['date','precipitation'])

df['date'] = pd.to_datetime(df['date'], format="%Y/%m/%d").dt.round("D")

prcp_df = df.set_index("date")
fig, ax= plt.subplots()
ax.bar(prcp_df.index,prcp_df['precipitation'])
monthloc = mdates.MonthLocator(bymonth=None, bymonthday=24, interval=1)
ax.legend(['Precipitation'],loc="best")
ax.xaxis.set_major_locator(monthloc)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
fig.autofmt_xdate()
plt.rcParams["font.size"] = 7
plt.title("Precipitation between 2016-8-24 to 2017-8-23")
plt.xlabel("Date")
plt.ylabel("Precipitation(inch)")

plt.tight_layout()
plt.savefig('Images/1Bar_Precipitation.png')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

,precipitation
count,365.000000
mean,0.169987
std,0.295722
min,0.000000
25%,0.008571
50%,0.070000
75%,0.191667
max,2.380000


In [18]:
# Design a query to show how many stations are available in this dataset?
stations=session.query(Station.station, Station.name).all()
len(stations)

9

In [25]:
# What are the most active stations? (i.e. what stations have the most rows)?
Station_counts = session.query(Measurement.station,Station.name,func.count(Measurement.station)).filter(Station.station == Measurement.station).filter(Measurement.date >= '2016-08-24').group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
# List the stations and the counts in descending order.
station_df = pd.DataFrame(Station_counts,columns=['id','name','Data_count'])
station_df

,id,name,Data_count
0,USC00519397,"WAIKIKI 717.2, HI US",360
1,USC00519281,"WAIHEE 837.5, HI US",351
2,USC00516128,"MANOA LYON ARBO 785.2, HI US",348
3,USC00513117,"KANEOHE 838.1, HI US",342
4,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",336
5,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",304
6,USC00517948,"PEARL CITY, HI US",182


In [26]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
lowest_temp = session.query(Measurement.station, func.min(Measurement.tobs)).filter(Measurement.station == "USC00519397").group_by(Measurement.station).all()
highest_temp = session.query(Measurement.station, func.max(Measurement.tobs)).filter(Measurement.station == "USC00519397").group_by(Measurement.station).all()
mean_temp = session.query(Measurement.station, func.avg(Measurement.tobs)).filter(Measurement.station == "USC00519397").group_by(Measurement.station).all()
# highest temperature recorded, and average temperature of the most active station?

print(lowest_temp, highest_temp, mean_temp)

[('USC00519397', 56.0)] [('USC00519397', 87.0)] [('USC00519397', 74.55323054331865)]


In [27]:
# Choose the station with the highest number of temperature observations.
highest_temp_stations = session.query(Measurement.station, func.max(Measurement.tobs),func.count(Measurement.tobs)).group_by(Measurement.station).all()
print(highest_temp_stations)

# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_station = session.query(Measurement.tobs).filter(Measurement.date >= '2016-08-24').filter(Measurement.station == "USC00519397").all()

temp_list = [x[0] for x in temp_station]

fig1, ax1 = plt.subplots()
ax1 = plt.hist(temp_list, bins=12)
plt.title("Histgram: Temperature of Waikiki")
plt.xlabel("Temp(F)")

plt.tight_layout()
plt.savefig('Images/2Hist_Waikiki_temp.png')

[('USC00511918', 87.0, 1979), ('USC00513117', 85.0, 2709), ('USC00514830', 85.0, 2202), ('USC00516128', 84.0, 2612), ('USC00517948', 87.0, 1372), ('USC00518838', 83.0, 511), ('USC00519281', 85.0, 2772), ('USC00519397', 87.0, 2724), ('USC00519523', 85.0, 2669)]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Bonus Challenge Assignment

In [28]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [70]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
last_year = calc_temps('2016-02-01', '2016-02-07')

In [71]:
# Plot the results from your previous query as a bar chart. 

fig2, ax2 = plt.subplots()
ax2.bar(["avg Temp"],height=[last_year[0][1]], yerr=(last_year[0][2]-last_year[0][0])/2, width=0.5)
ax2.set_xlim(-1, 1)
ax2.set_ylabel("Temp(F)")
ax2.set_title("Trip Avg Temp")
plt.tight_layout()
plt.savefig('Images/3bar_Trip_avg_temp.png')
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.

total_rainfall = session.query(Measurement.station,Station.name,func.sum(Measurement.prcp)).filter(Station.station == Measurement.station).filter(Measurement.date >= '2016-02-01').filter(Measurement.date <= '2016-02-07').group_by(Measurement.station).order_by(func.sum(Measurement.prcp).desc()).all()
total_rainfall
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



[('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 0.9299999999999999),
 ('USC00513117', 'KANEOHE 838.1, HI US', 0.82),
 ('USC00519281', 'WAIHEE 837.5, HI US', 0.78),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 0.76),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 0.42000000000000004),
 ('USC00519397', 'WAIKIKI 717.2, HI US', 0.08),
 ('USC00517948', 'PEARL CITY, HI US', None)]

In [54]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [95]:
# calculate the daily normals for your trip
from datetime import datetime
from datetime import timedelta

start_date = "2018-02-01"
end_date ="2018-02-08"

start = datetime.strptime(start_date, '%Y-%m-%d').date()
end   = datetime.strptime(end_date, '%Y-%m-%d').date()

def daterange(_start, _end):
    for n in range((_end - _start).days):
        yield _start + timedelta(n)

trip_dates = []
format_trip_dates = []

for i in daterange(start, end):
    trip_dates.append(i.strftime('%y-%m-%d'))
    format_trip_dates.append(i.strftime('%m-%d'))

normals =[]
for day in format_trip_dates:
    normals.append(daily_normals(day)[0])

normals

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


[(56.0, 68.14035087719299, 78.0),
 (59.0, 68.85964912280701, 77.0),
 (58.0, 67.79661016949153, 77.0),
 (61.0, 68.14035087719299, 76.0),
 (59.0, 69.6842105263158, 80.0),
 (58.0, 70.38983050847457, 77.0),
 (60.0, 69.0, 77.0)]

In [99]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.DataFrame(normals,columns=["TMIN","TAVG","TMAX"])
normals_df['date'] = trip_dates

normals_df = normals_df.set_index('date')
normals_df

,TMIN,TAVG,TMAX
date,,,
18-02-01,56.0,68.140351,78.0
18-02-02,59.0,68.859649,77.0
18-02-03,58.0,67.796610,77.0
18-02-04,61.0,68.140351,76.0
18-02-05,59.0,69.684211,80.0
18-02-06,58.0,70.389831,77.0
18-02-07,60.0,69.000000,77.0


In [103]:
# Plot the daily normals as an area plot with `stacked=False`
normals_df.plot.area(stacked=False)
plt.title("Trip dates Temp information")
plt.tight_layout()
plt.savefig('Images/4area_Trip_dates_temp.png')

C:\Users\khiro\anaconda3\lib\site-packages\pandas\plotting\_matplotlib\core.py:320: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …